# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [58]:
# abrir o navegador
from selenium import webdriver
navegador = webdriver.Chrome()
from selenium.webdriver.common.keys import Keys

# entrar no google
navegador.get("https://www.google.com/")

# pesquisar "cotação do dólar"
# para achar o xpath: clicar com o botão direito na página do navegador -> inspecionar
# no topo do código exibido, ir no ícone de seta, clicar no local desejado -> botão direito no código copiar -> copiar xpath
# colar xpath no código abaixo
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

# dar um 'enter'
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar o número que aparece no resultado do google
# dentro do inspecionar, procurar a informação procurada
# no caso queremos pegar o data-value
cotação_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print("a cotação do dólar é: ", cotação_dolar)

# fazer o mesmo processo para cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotação_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print("a cotação do euro é: ", cotação_euro)

# o mesmo para a cotação do ouro
# o google não exibe a cotação do ouro, por isso vamos entrar no site https://www.melhorcambio.com/ouro-hoje e repetir os procedimentos
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotação_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotação_ouro = cotação_ouro.replace(",", ".")
print("a cotação do ouro é: ", cotação_ouro)

# fechar o navegador
navegador.quit()

a cotação do dólar é:  5.219799999999999
a cotação do euro é:  6.1275649780000006
a cotação do ouro é:  294.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [59]:
# importar e atualizar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [60]:
# atualizando a cotação
# tabela.loc[linha, coluna]  #loc para localizar 
# aonde a coluna moeda é igual a 'Dólar'
tabela.loc[tabela["Moeda"] == "Dólar" ,"Cotação"] = float(cotação_dolar)  
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotação_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotação_ouro)
display(tabela)

# atualizar o preço de compra => 'Preço Base Original' * 'Cotação'
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

# atualizar o preço de venda => 'Preço Base Reais' * 'Margem'
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

# editar a tabela 'Preço Final' - Arredondando o valor
#tabela["Preço Final"] = tabela["Preço Final"].map("{:.1f}".format)

# mudando nome das colunas
tabela = tabela.rename(columns={'Preço Base Reais': 'Preço de Compra'})
tabela = tabela.rename(columns={'Preço Final': 'Preço de Venda'})


display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.219800,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.127565,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.219800,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.219800,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.127565,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.219800,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,294.120000,7000.00,1.15,8050.000


,Produtos,Preço Base Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.219800,5219.747802,1.40,7307.646923
1,Carro Renault,4500.00,Euro,6.127565,27574.042401,2.00,55148.084802
2,Notebook Dell,899.99,Dólar,5.219800,4697.767802,1.70,7986.205263
3,IPhone,799.00,Dólar,5.219800,4170.620200,1.70,7090.054340
4,Carro Fiat,3000.00,Euro,6.127565,18382.694934,1.90,34927.120375
5,Celular Xiaomi,480.48,Dólar,5.219800,2508.009504,2.00,5016.019008
6,Joia 20g,20.00,Ouro,294.120000,5882.400000,1.15,6764.760000


### Agora vamos exportar a nova base de preços atualizada

In [61]:
tabela.to_excel("Produtos_Novo.xlsx", index = False)  # index = False retira o índice